In [1]:
import tensorflow as tf

print(tf.__version__)

2021-09-06 21:10:38.009280: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-06 21:10:38.009315: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2.5.0


# Features and Example protocol buffer

In [5]:
with open('story.txt') as f:
  words = f.read().split()

word_bytes = [word.encode() for word in words]
words_feature = tf.train.Feature(
  bytes_list=tf.train.BytesList(value=word_bytes)
)
# print(repr(words_feature))

In [6]:
# Convert a dictionary of different features into an Example
def dict_to_example(data_dict, config):
    feature_dict = {}
    for feature_name, value in data_dict.items():
        feature_config = config[feature_name]
        shape = feature_config['shape']
        if shape == () or shape == []:
            value = [value]
        value_type = feature_config['type']
        if value_type == 'int':
            feature_dict[feature_name] = make_int_feature(value)
        elif value_type == 'float':
            feature_dict[feature_name] = make_float_feature(value)
        elif value_type == 'string' or value_type == 'bytes':
            feature_dict[feature_name] = make_bytes_feature(
                value, value_type)
    features = tf.train.Features(feature=feature_dict)
    return tf.train.Example(features=features)


def make_int_feature(value):
    int_f = tf.train.Feature(
        int64_list=tf.train.Int64List(value=value)
    )
    return int_f


def make_float_feature(value):
    return tf.train.Feature(
        float_list=tf.train.FloatList(value=value)
    )


def make_bytes_feature(value, value_type):
    # CODE HERE
    if value_type == 'string':
        value = [v.encode() for v in value]
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(
            value=value,
        )
    )


In [7]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    # BytesList won't unpack a string from an EagerTensor.
    value = value.numpy()
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))


def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


# TFRecord

In [12]:

print(_bytes_feature('test_string'.encode()))
print(_int64_feature(1))


bytes_list {
  value: "test_string"
}

int64_list {
  value: 1
}



In [15]:
def serialize_example(feature0, feature1, feature2, feature3):
  """
  Creates a tf.train.Example message ready to be written to a file.
  """
  # Create a dictionary mapping the feature name to the tf.train.Example-compatible
  # data type.
  feature = {
      'feature0': _int64_feature(feature0),
      'feature1': _int64_feature(feature1),
      'feature2': _bytes_feature(feature2),
      'feature3': _float_feature(feature3),
  }

  # Create a Features message using tf.train.Example.

  example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
  return example_proto.SerializeToString()

encoded = serialize_example(False, 4, b'goat', 0.2332) # return a byte string to be written to a file
print(encoded)
print()
decoded = tf.train.Example.FromString(encoded)
print(decoded)

b'\nR\n\x14\n\x08feature2\x12\x08\n\x06\n\x04goat\n\x11\n\x08feature0\x12\x05\x1a\x03\n\x01\x00\n\x14\n\x08feature3\x12\x08\x12\x06\n\x04\xfb\xcbn>\n\x11\n\x08feature1\x12\x05\x1a\x03\n\x01\x04'

features {
  feature {
    key: "feature0"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "feature1"
    value {
      int64_list {
        value: 4
      }
    }
  }
  feature {
    key: "feature2"
    value {
      bytes_list {
        value: "goat"
      }
    }
  }
  feature {
    key: "feature3"
    value {
      float_list {
        value: 0.23319999873638153
      }
    }
  }
}



In [19]:
filename = 'test.tfrecord'
writer = tf.io.TFRecordWriter(filename)
writer.write(encoded)
writer.close()

# Reading and writing data with tf.data